In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
Dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
Dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
Dataset=pd.get_dummies(Dataset,dtype=int,drop_first=True)

In [5]:
Dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
Dataset=Dataset.drop("User ID",axis=1)

In [7]:
Dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [8]:
Dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [9]:
Dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [10]:
independent=Dataset[[ 'Age', 'EstimatedSalary', 'Gender_Male']]

In [11]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [12]:
independent.shape

(400, 3)

In [13]:
dependent=Dataset[['Purchased']]

In [14]:
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [15]:
# splitting test and train data
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [16]:
X_train

,Age,EstimatedSalary,Gender_Male
218,46,96000,0
101,28,59000,1
311,39,96000,1
194,28,89000,1
326,41,72000,1
...,...,...,...
323,48,30000,0
192,29,43000,1
117,36,52000,1
47,27,54000,0


In [17]:
X_test

,Age,EstimatedSalary,Gender_Male
132,30,87000,1
309,38,50000,0
341,35,75000,1
196,30,79000,0
246,35,50000,0
...,...,...,...
168,29,148000,1
150,26,15000,0
393,60,42000,1
66,24,19000,1


In [18]:
Y_train

,Purchased
218,0
101,0
311,1
194,0
326,0
...,...
323,1
192,0
117,0
47,0


In [19]:
Y_test

,Purchased
132,0
309,0
341,0
196,0
246,0
...,...
168,1
150,0
393,1
66,0


In [20]:
y_squeezed=np.squeeze(Y_train)
y_squeezed
# this step can be done in dependent dataset

218    0
101    0
311    1
194    0
326    0
      ..
323    1
192    0
117    0
47     0
172    0
Name: Purchased, Length: 266, dtype: int64

In [62]:
# model creation
# X_train.values are used to resolve the feature warning thrown when we try to predict the result by passing x input
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier(criterion='entropy',n_estimators=10,random_state=0)
classifier.fit(X_train.values,y_squeezed)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [64]:
Y_test.value_counts()

Purchased
0            85
1            49
Name: count, dtype: int64

In [66]:
y_pred=classifier.predict(X_test.values)

In [68]:
# evaluation metrics for classification is confusion matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test,y_pred)


In [70]:
#diagonal represent positive classification:
# Not purchase classified correctly-78 out of 85
# purchased classified correctly 43 out of 49
print(cm) 

[[78  7]
 [ 6 43]]


In [72]:
# classification report to get the values of precision, recall and accuracy
from sklearn.metrics import classification_report
clf_report=classification_report(Y_test,y_pred)

In [74]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.93      0.92      0.92        85
           1       0.86      0.88      0.87        49

    accuracy                           0.90       134
   macro avg       0.89      0.90      0.90       134
weighted avg       0.90      0.90      0.90       134



In [76]:
# save the model
import pickle
filename="RandomForest_final.sav"
pickle.dump(classifier,open(filename,"wb"))

In [78]:
#loaded the model
loaded_model=pickle.load(open(filename,"rb"))
result=loaded_model.predict([[20,20000,1]])

In [80]:
print(result)

[0]


In [82]:
result=loaded_model.predict([[50,30000,1]])

In [84]:
print(result)

[1]
